In [ ]:
!pip install ta
!pip install scikeras

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=72c52c022dcd0dda8586e1133abe878dfdebf3e2bbb5b9beba5b4e2bbd5c801d
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [ ]:
!pip install scikit-learn==1.4.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 43.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


In [ ]:
import ta
import yfinance as yf
n50 = yf.Ticker("^NSEI")

n50 = n50.history(period = "max")

In [ ]:
del n50["Dividends"]
del n50["Stock Splits"]
n50 = n50[n50["Volume"]!=0]

In [ ]:
n50["tomorrow"] = n50["Close"].shift(-1)
n50["target"] = (n50["tomorrow"]>n50["Close"]).astype(int)

'''
n50["RSI"] = ta.momentum.rsi(n50["Close"], window = 14)

macd = ta.trend.MACD(n50["Close"], window_slow=26, window_fast=12, window_sign=9)
n50["MACD"] = macd.macd() #calculates main macd line, confusing variable name
n50["MACD_signal"] = macd.macd_signal()

bb = ta.volatility.BollingerBands(close=n50["Close"], window=20, window_dev=2)
n50["BB_high"] = bb.bollinger_hband()
n50["BB_low"] = bb.bollinger_lband()

n50["EMA_F"] = ta.trend.ema_indicator(close = n50["Close"], window = 50)
n50["EMA_S"] = ta.trend.ema_indicator(close = n50["Close"], window = 200)
'''

In [ ]:
n50 = n50.dropna()

In [ ]:
n50

,Open,High,Low,Close,Volume,tomorrow,target
Date,,,,,,,
2013-01-21 00:00:00+05:30,0.040557,0.037055,0.044990,0.038092,0.072229,6048.500000,0
2013-01-22 00:00:00+05:30,0.040291,0.037386,0.043820,0.036477,0.071180,6054.299805,1
2013-01-23 00:00:00+05:30,0.038992,0.035883,0.042900,0.036754,0.075598,6019.350098,0
2013-01-24 00:00:00+05:30,0.038675,0.035669,0.042268,0.035084,0.102214,6074.649902,1
2013-01-25 00:00:00+05:30,0.037643,0.036396,0.042582,0.037726,0.081451,6074.799805,1
...,...,...,...,...,...,...,...
2025-03-27 00:00:00+05:30,0.866080,0.874478,0.869764,0.874631,0.281738,23519.349609,0
2025-03-28 00:00:00+05:30,0.874001,0.874609,0.871570,0.871163,0.163068,23165.699219,0
2025-04-01 00:00:00+05:30,0.861662,0.870599,0.856651,0.854267,0.173505,23332.349609,1


In [ ]:
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import precision_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.layers import Input

from scikeras.wrappers import KerasClassifier

scaler  = MinMaxScaler()
predictors_n50 = ["Volume", "High", "Close", "Open", "Low"]

horizons = [2, 5, 60, 260, 1000]
#new_predictors = []

for horizon in horizons:
  rolling_avgs = n50.rolling(horizon).mean()

  ratio_col = f"Close_Ratio_{horizon}"
  n50[ratio_col] = n50["Close"] / rolling_avgs["Close"]

  trend_col = f"Trend_{horizon}"
  n50[trend_col] = n50.shift(1).rolling(horizon).sum()["target"]

  predictors_n50 += [ratio_col, trend_col]

n50 = n50.dropna()
n50[predictors_n50] = scaler.fit_transform(n50[predictors_n50])

<ipython-input-58-d740cf226bbf>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n50[ratio_col] = n50["Close"] / rolling_avgs["Close"]
<ipython-input-58-d740cf226bbf>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n50[trend_col] = n50.shift(1).rolling(horizon).sum()["target"]
<ipython-input-58-d740cf226bbf>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [ ]:
import numpy as np

def sequences(data, target, time_steps = 10):
  data_seq = []
  target_seq = []
  for i in range(len(data) - time_steps):
    data_seq.append(data.iloc[i:(i+time_steps)].values)
    target_seq.append(target.iloc[i+time_steps])
  return np.array(data_seq), np.array(target_seq)

In [ ]:
def create_model(units = 150, dropout_rate = 0.1, batch_size = 128, epochs = 10):
  model = Sequential()
  model.add(Input(shape = (x_train.shape[1], x_train.shape[2])))
  model.add(LSTM(units, return_sequences = True))
  model.add(Dropout(dropout_rate))
  model.add(LSTM(units = units))
  model.add(Dropout(dropout_rate))
  model.add(Dense(units = 1, activation = "sigmoid"))
  model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
  return model

In [ ]:
x, y = sequences(n50[predictors_n50], n50["target"], time_steps = 10)

x_train_val, x_test, y_train_val, y_test = train_test_split(x, y, test_size=0.1, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.1, random_state=1)

In [ ]:
model = KerasClassifier(model=create_model, units=200, dropout_rate=0.1, batch_size=128, epochs=10)
results = model.fit(x_train, y_train, validation_data=(x_val, y_val))

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 188ms/step - accuracy: 0.5313 - loss: 0.6919 - val_accuracy: 0.4368 - val_loss: 0.7341
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 0.4959 - loss: 0.7080 - val_accuracy: 0.5632 - val_loss: 0.6773
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 0.5189 - loss: 0.7108 - val_accuracy: 0.5977 - val_loss: 0.6798
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - accuracy: 0.5039 - loss: 0.6916 - val_accuracy: 0.4368 - val_loss: 0.7030
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - accuracy: 0.4585 - loss: 0.6969 - val_accuracy: 0.5632 - val_loss: 0.6900
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 0.5002 - loss: 0.6931 - val_accuracy: 0.5977 - val_loss: 0.6871
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 148ms/step - accuracy: 0.5219 - loss: 0.6910 - val_accuracy: 0.5632 - val_loss: 0.6781
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 196ms/step - accuracy: 0.5223 - loss: 0.6924 - val_accuracy: 0.5632 - val_loss:

In [ ]:
accuracy = model.score(x_test, y_test)
print(f"accuracy: {accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step
accuracy: 0.5416666666666666


In [ ]:
predictions = model.predict(x_test)
precision = precision_score(y_test, predictions)
print(f"precision: {precision}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
precision: 0.5416666666666666


In [ ]:
prec = 0

for i in range(len(predictions)):
    if predictions[i] == y_test[i]:
      prec = prec+1\

print(prec/len(y_test))

0.5416666666666666


In [ ]:
from sklearn.metrics import recall_score, f1_score

recall = recall_score(y_test, predictions)
print(f"Recall: {recall}")

f1 = f1_score(y_test, predictions)
print(f"F1 Score: {f1}")

Recall: 1.0
F1 Score: 0.7027027027027027


In [ ]:
param_grid = {
    'model__units': [50, 100, 150],
    'model__dropout_rate': [0.1, 0.2, 0.3],
    'batch_size': [32, 64, 128],
    'epochs': [10, 20, 30]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(x_train, y_train)

_, accuracy = grid_result.best_estimator_.model_.evaluate(x_test, y_test, verbose=0)

print(f"Best Accuracy: {accuracy}")
print(f"Best Parameters: {grid_result.best_params_}")

#output with indicators came to be 150 units, 0.1 dropout rate, 10 epochs, 128 batch size

Epoch 1/10


KeyboardInterrupt: 